# BUILD MEL

In [1]:
import numpy as np
import librosa
import pandas as pd
from sklearn import preprocessing
from tqdm.notebook import trange, tqdm

### GLOBAL VARIABLES

In [2]:
INPUT_PATH = 'data/processed/submission.npy'
OUTPUT_PATH = 'data/features/submission_mel_005.npy'
SR = 22050

### FUNCTIONS

In [3]:
def wave2mel(wave, sr, n_fft, hop_length, n_mels, max_pad_len, normalize, scaling, padding_mode, **kwargs):

    #wave, sr = librosa.load(wav_file, mono=True)
    
    #if 0 < len(wave): # workaround: 0 length causes error
    #    wave, _ = librosa.effects.trim(wave) # trim, top_db=default(60)

    if normalize:
        wave = librosa.util.normalize(wave) # normalizing data before mel

    # making melspect from signal
    S = librosa.feature.melspectrogram(y=wave, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, fmax=sr//2, **kwargs)

    # scaling
    if scaling:
        S = preprocessing.scale(S, axis=1)

    if max_pad_len:
        if S.shape[1] > max_pad_len:
            S = S[:,:max_pad_len]
        else:
            pad_width = max_pad_len - S.shape[1]
            S = np.pad(S, pad_width=((0, 0), (0, pad_width)), mode=padding_mode)
    
    S = librosa.power_to_db(S)
    S = S.astype(np.float32)
   
    return S

In [4]:
def gen_mel(input_path, output_path, sr, n_fft=2048, hop_length=512, n_mels=128, max_pad_len=None, normalize=True, scaling=False, padding_mode='constant', **kwargs):
    
    # Load wave files
    wavfiles = np.load(input_path)
    
    # Init Mel vectors
    X_mel = []
    
    for idx in trange(wavfiles.shape[0]):
        X_mel.append(wave2mel(wavfiles[idx, :], sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, max_pad_len=max_pad_len, 
                             normalize=normalize, scaling=scaling, padding_mode=padding_mode, **kwargs))
        pass
    
    X_mel = np.array(X_mel)
    
    # Saving Mels 
    np.save(output_path, X_mel)
    pass

### RUNING

In [5]:
gen_mel(INPUT_PATH, OUTPUT_PATH, SR, n_fft=441, hop_length=110, n_mels=35, scaling=True, normalize=True)